In [1]:
import math
import numpy as np
from shapely.geometry import MultiPoint

In [2]:
breakpoints = [([-37.647863, 363.520713], 1), ([-86.993307, 566.306022], 1), ([-116.276208, 686.644227], 1), ([-43.256272, 704.133424], 0), ([-97.187671, 928.036095], 0), ([-30.823008, 943.939461], 1), ([-22.791816, 910.584813], 0), ([41.779946, 926.056381], 0), ([129.221734, 546.278255], 1)]

Cxy = list(range(58, 109))
Ch = [9, 12]

In [3]:
class BaseSection:
    def __init__(self, angle, leng, widt, road):
        self.angle = angle
        self.an_normalized = self.angle_normalized()
        self.length, self.width, self.road = leng, widt, road

    def angle_normalized(self):
        if self.angle > math.pi / 4:
            if self.angle < math.pi / 2:
                ang = (math.pi / 2) - self.angle
            elif 3 * math.pi / 4 > self.angle > math.pi / 2:
                ang = self.angle - (math.pi / 2)
            else:
                ang = math.pi - self.angle
        else:
            ang = self.angle
        return ang

In [173]:
class PolyClusters:
    def __init__(self, split_points, cells, road):
        self.split_points = split_points
        self.cells = cells
        self.road = road
        self.bound_rec = np.asarray(MultiPoint([i[0] for i in self.split_points]).minimum_rotated_rectangle.exterior.coords)
        self.projection = self.points_projected()
        self.distances, self.coordinates = self.distances()

    #считаем точки спроецированные на баундинг ректангл
    def points_projected(self):
        points = []
        for i, v in enumerate(self.bound_rec[0:4]):
            if i != len(self.bound_rec)-1:
                temp =[]
                length = np.sum((v-self.bound_rec[i+1])**2)
                for sp in self.split_points:
                    if sp[1] == 1:
                        t = max(0, min(1, np.sum((np.asarray(sp[0]) - v) * (self.bound_rec[i+1] - v)) / length))
                        projection = np.asarray(v + t * (self.bound_rec[i+1] - v))
                        temp.append(projection)
                    else:
                        pass
                points.append(np.asarray(temp))
        return np.asarray(points)



    #считпем расстояния между точками и убиваем мелкие недоразумения вне зависимости от веса. Сейчас 58, возможно надо сделать ключем
    #возвращаем длины и тюпли точек из которых можно сразу считать вектора
    def distances(self):
        arr = np.delete(self.projection, -1, axis=1)
        shift_arr = np.delete(np.roll(self.projection, -1, axis=1), -1, axis=1)
        dist = np.linalg.norm(arr-shift_arr, axis = 2 )
        values = []
        distance = []

        for p, i in enumerate(dist):
            values_to_del = []
            d =[]
            sum = 0
            for v in range(len(i)):
                if i[v] < 58 and v!=len(i)-1:
                    sum+=i[v]
                    values_to_del.append(v+1)
                elif i[v] < 58 and v==len(i)-1:
                    sum+=i[v]
                    d[-1]+=sum
                    values_to_del.append(v)
                else:
                    d.append(sum + i[v])
                    sum = 0
            values.append(d)
            clear_list = np.delete(self.projection[p], values_to_del, axis=0)
            distance.append(np.stack((np.delete(clear_list, -1, axis=0), np.delete(np.roll(clear_list, -1, axis=0), -1, axis=0)), axis = 1))

        return values, np.asarray(distance)


    #комбинации ячеек
    def grid_options(self):
        ans = []
        temp = []
        wd = round(length[0])
        def find_numbers(ans, values, r, temp, wd, index):
            if wd == 0:
                ans.append(list(temp))
            for i in range(index, len(values)):
                if (wd - r-values[i]) >= 0:
                    temp.append([r//2,values[i],r//2])
                    find_numbers(ans, values, r, temp, wd - r -values[i], i)
                    temp.remove([r//2,values[i],r//2])

        find_numbers(ans, values, r, temp, wd, 0)
        combinations = []
        for i in ans:
            combinations.append([item for sublist in i for item in sublist])
        return combinations

    #функция просчета ячеек будет внутри расчета координат, чтобы не терять структуру
    def coordinates(self):








In [176]:
p_c = PolyClusters(breakpoints, Cxy)
v = p_c.distances
#v = p_c.projection
#print(p_c.projection)
print(v)

[[208.6867483082789, 123.84027628145742, 231.56154456452532, 423.4319528758201], [145.12783227425433, 66.73930987798941], [208.68674830827885, 123.84027628145742, 231.5615445645252, 423.43195287581995], [145.12783227425442, 66.73930987798934]]


/var/folders/l7/52t7xngd2973k8x9_y55rmyh0000gn/T/ipykernel_14306/2205715497.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return values, np.asarray(distance)


In [216]:
ii = [208.68, 123.84, 231.56, 423.43, 146]
cx = list(range(58,109))
r = 40

def grid_options(length, values, r):
    ans = []
    temp = []
    wd = round(length[0])
    def find_numbers(ans, values, r, temp, wd, index):
        if wd == 0:
            ans.append(list(temp))
        for i in range(index, len(values)):
            if (wd - r-values[i]) >= 0:
                temp.append([r//2,values[i],r//2])
                find_numbers(ans, values, r, temp, wd - r -values[i], i)
                temp.remove([r//2,values[i],r//2])

    find_numbers(ans, values, r, temp, wd, 0)
    combinations = []
    for i in ans:
        combinations.append([item for sublist in i for item in sublist])
    return combinations

v = grid_options(ii,cx,r)
v

[[20, 58, 20, 20, 71, 20],
 [20, 59, 20, 20, 70, 20],
 [20, 60, 20, 20, 69, 20],
 [20, 61, 20, 20, 68, 20],
 [20, 62, 20, 20, 67, 20],
 [20, 63, 20, 20, 66, 20],
 [20, 64, 20, 20, 65, 20]]